In [37]:
# Imports
import os
from django.apps import apps as django_apps
from edc_appointment.models import Appointment
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [38]:
replacements = SubjectConsent.objects.filter(
    Q(subject_identifier__startswith='BR') | Q(subject_identifier__startswith='CR')).values_list(
    'subject_identifier', flat=True)
sidxs = list(set(replacements))

In [39]:
# Dataset, Screening, Caregiver Enrol Info, Locator, Subject Consent, RegisteredSubject,
# Child Registered Subject, Schedules, Appointments, Visits, CRFs and Requisitions

In [40]:
def get_model_objs(subject_identifier, model_cls):
    model_objs = model_cls.objects.filter(subject_identifier=subject_identifier)
    return model_objs

In [41]:
def get_onschedule_obj(subject_identifier, model_cls):
    onschedules = model_cls.objects.filter(subject_identifier=subject_identifier)
    return onschedules

In [42]:
def get_crf_requisition(subject_identifier, model_cls):
    crf_requisition = model_cls.objects.filter(maternal_visit__subject_identifier=subject_identifier)
    return crf_requisition

In [43]:
def get_references(subject_identifier):
    return Reference.objects.filter(identifier=subject_identifier)

In [44]:
def get_action_items(subject_identifier):
    return ActionItem.objects.filter(subject_identifier=subject_identifier)

In [51]:
for sidx in sidxs:
    new_sidx = sidx.replace('B', '') if sidx.startswith('B') else sidx.replace('C', '')

    datasets = get_model_objs(sidx, MaternalDataset).update(subject_identifier=new_sidx)
    screenings = get_model_objs(sidx, ScreeningPriorBhpParticipants).update(subject_identifier=new_sidx)
    enrol_infos = get_model_objs(sidx, CaregiverPreviouslyEnrolled).update(subject_identifier=new_sidx)
    locators = get_model_objs(sidx, CaregiverLocator).update(subject_identifier=new_sidx)
    consents = get_model_objs(sidx, SubjectConsent).update(subject_identifier=new_sidx)
    caregiver_reg = get_model_objs(sidx, RegisteredSubject).update(subject_identifier=new_sidx)

    child_reg = RegisteredSubject.objects.filter(relative_identifier=sidx).update(relative_identifier=new_sidx)

    schedules = SubjectScheduleHistory.objects.filter(subject_identifier=sidx)

    onschedules = []
    for schedule in schedules:
        onschedule_model_cls = django_apps.get_model(schedule.onschedule_model)
        get_onschedule_obj(sidx, onschedule_model_cls).update(subject_identifier=new_sidx)
    schedules.update(subject_identifier=new_sidx)

    appts = Appointment.objects.filter(subject_identifier=sidx).update(subject_identifier=new_sidx)
    visits = MaternalVisit.objects.filter(subject_identifier=sidx).update(subject_identifier=new_sidx)

    crf_metadata = CrfMetadata.objects.filter(subject_identifier=sidx)

    keyed_crf_metadata = crf_metadata.filter(entry_status='KEYED').values_list('model', flat=True)
    keyed_crf_metadata = list(set(keyed_crf_metadata))
    crf_objs = []
    for crf in keyed_crf_metadata:
        crf_model_cls = django_apps.get_model(crf)
        crf_objs.append(get_crf_requisition(sidx, crf_model_cls))
    crf_metadata.update(subject_identifier=new_sidx)

    requisition_metadata = RequisitionMetadata.objects.filter(subject_identifier=sidx)

    keyed_req_metadata = requisition_metadata.filter(entry_status='KEYED').values_list('model', flat=True)
    keyed_req_metadata = list(set(keyed_req_metadata))
    req_objs = []
    for requisition in keyed_req_metadata:
        req_model_cls = django_apps.get_model(requisition)
        req_objs.append(get_crf_requisition(sidx, req_model_cls))
    requisition_metadata.update(subject_identifier=new_sidx)

    references = get_references(sidx).update(identifier=new_sidx)
    action_items = get_action_items(sidx).update(subject_identifier=new_sidx)

    print(datasets)
    print(screenings)
    print(enrol_infos)
    print(locators)
    print(consents)
    print(caregiver_reg)
    print(child_reg)
    print(schedules)
    print(onschedules)
    print(appts)
    print(visits)
    print(crf_metadata)
    print(crf_objs)
    print(requisition_metadata)
    print(req_objs)
    print(references)
    print(action_items)

1
1
1
1
1
1
1
<QuerySet []>
[]
9
4
<QuerySet []>
[]
<QuerySet []>
[]
20
0
1
1
1
1
1
1
1
<QuerySet []>
[]
11
6
<QuerySet []>
[]
<QuerySet []>
[]
38
0
1
1
1
1
1
1
1
<QuerySet []>
[]
9
4
<QuerySet []>
[]
<QuerySet []>
[]
16
0
1
1
1
1
1
1
1
<QuerySet []>
[]
10
5
<QuerySet []>
[]
<QuerySet []>
[]
21
0
1
1
1
1
1
1
1
<QuerySet []>
[]
8
3
<QuerySet []>
[]
<QuerySet []>
[]
17
0


In [61]:
pids = ['R142-040990631-9', 'R142-040990642-6', 'R142-040990257-3', 'R142-040990903-2', 'R142-040990227-6']
for pid in pids:
    schedules = SubjectScheduleHistory.objects.filter(subject_identifier=pid)
    for schedule in schedules:
        onschedule_model_cls = django_apps.get_model(schedule.onschedule_model)
        get_onschedule_obj(f'C{schedule.subject_identifier}', onschedule_model_cls).update(
            subject_identifier=schedule.subject_identifier)
        get_onschedule_obj(f'B{schedule.subject_identifier}', onschedule_model_cls).update(
            subject_identifier=schedule.subject_identifier)

In [62]:
OnScheduleCohortBSec.objects.filter(subject_identifier='R142-040990642-6')

<QuerySet [<OnScheduleCohortBSec: R142-040990642-6 2022-12-14>]>